In [69]:
import time
import cmath
import json
from collections import defaultdict
from typing import List, Dict, Callable

import numpy as np
import qutip as qtp
from matplotlib import pyplot as plt

from pysqkit import qubits, drives, couplers
from pysqkit import QubitSystem, Qubit, Coupling
from pysqkit.util.linalg import get_mat_elem
from pysqkit.util.phys import temperature_to_thermalenergy

In [70]:
def dressed_state(
    lev: str,
    hamil: qtp.qobj.Qobj,
    lev_t: int,
    lev_f: int
):
    """
    Description
    ---------------------------------------------------------
    Function that returns the dressed state
    """
    
    #eig_en_0, eig_states_0 = h_0.eigenstates()
    eig_en, eig_states = hamil.eigenstates()
    
    lev_list = [int(index) for index in lev]
    
    if lev_list[0] >= lev_t or lev_list[1] >= lev_f:
        raise ValueError("Index error: indeces out of bound")
    state_0 = qtp.tensor(qtp.basis(lev_t, lev_list[0]), qtp.basis(lev_f, lev_list[1]))
    #en_0 = eig_en_0[state_index]
    eig_states_list = list(eig_states)
    state = eig_states_list[0]
    en = eig_en[0]
    fid = qtp.fidelity(state*state.dag(), state_0*state_0.dag())
    for state_tmp, en_tmp in zip(eig_states_list, eig_en):
        fid_tmp = qtp.fidelity(state_tmp*state_tmp.dag(), state_0*state_0.dag())
        if fid_tmp > fid:
            state = state_tmp
            fid = fid_tmp
            en = en_tmp
    loc = np.argmax(np.abs(np.array(state)) )
    phase = cmath.phase(state[loc])
    state = np.exp(-1j*phase)*state
    return en, state

def projector(
    hamil: qtp.qobj.Qobj,
    lev_t: int,
    lev_f: int,
    labels=["00", "01", "10", "11"]
):
    states = {}
    proj = 0
    for label in labels:
        states[label] = dressed_state(label, hamil, lev_t, lev_f)[1]
        proj += states[label]*states[label].dag()
    return proj, states

def h_sw_exact(
    proj_0: qtp.qobj.Qobj,
    proj: qtp.qobj.Qobj,
    hamil: qtp.qobj.Qobj,
    states_0: dict,
    lev_t: int,
    lev_f: int
):
    refl_0 = qtp.tensor(qtp.qeye(lev_t), qtp.qeye(lev_f)) - 2*proj_0
    refl = qtp.tensor(qtp.qeye(lev_t), qtp.qeye(lev_f)) - 2*proj
    u_sw = (refl_0*refl).sqrtm()
    h_sw = u_sw*hamil*u_sw.dag()
    h_eff = np.zeros([4, 4], dtype=complex)
    for key_row, row in zip(states_0.keys(), range(0, 4)):
        for key_col, col in zip(states_0.keys(), range(0, 4)):
            h_eff[row, col] = hamil.matrix_element(states_0[key_row], states_0[key_col])
    return h_eff

def get_pauli_coeff(
    op: np.ndarray,
    label: str
):
    pauli = {}
    pauli["I"] = np.identity(2, dtype=complex)
    pauli["X"] = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)
    pauli["Y"] = np.array([[0.0, -1j*1.0], [1j*1.0, 0.0]], dtype=complex)
    pauli["Z"] = np.array([[1.0, 0.0], [0.0, -1.0]], dtype=complex)
    label_list = [x for x in label]
    pauli_op = np.kron(pauli[label_list[0]], pauli[label_list[1]])
    coeff = 1/4*np.trace(op.conj().T.dot(pauli_op))
    return coeff

def get_params_sw(
    transm: Qubit,
    flx: Qubit,
    jc: float,
    drive_params: Dict
):
    """
    Description
    -----------------------------------------------------------------------
    Returns the exact cross-resonance coefficient in MHz.
    """
    
    if flx.dim_hilbert != 4:
        raise ValueError("Fluxonium dimension error: 4 levels of the fluxonium"
                        " must be included")
    if transm.dim_hilbert != 3:
        raise ValueError("Transmon dimension error: 3 levels of the fluxonium"
                        " must be included")
    
    freq_drive = drive_params["freq"]
    eps_drive = drive_params["eps"]
    phase_drive = np.pi/2
    
    eig_flx = flx.eig_energies()
    
    lev_t = 3
    lev_f = 4
    id_t = qtp.qeye(lev_t)
    id_f = qtp.qeye(lev_f)
    proj_1_t = qtp.basis(lev_t, 1)*qtp.basis(lev_t, 1).dag()
    proj_2_t = qtp.basis(lev_t, 2)*qtp.basis(lev_t, 2).dag()
    h_t_single = (transm.freq - freq_drive)*proj_1_t + (transm.anharm + 2*(transm.freq - freq_drive))*proj_2_t
    h_t = qtp.tensor(h_t_single, id_f)

    # Fluxonium Hamiltonian
    proj_1_f = qtp.basis(lev_f, 1)*qtp.basis(lev_f, 1).dag()
    proj_2_f = qtp.basis(lev_f, 2)*qtp.basis(lev_f, 2).dag()
    proj_3_f = qtp.basis(lev_f, 3)*qtp.basis(lev_f, 3).dag()
    ground_en_f = flx.eig_energies()[0]
    h_f_single = flx.hamiltonian(as_qobj=True) - ground_en_f*id_f - freq_drive*proj_1_f - \
        freq_drive*proj_3_f - 2*freq_drive*proj_2_f
    h_f = qtp.tensor(id_t, h_f_single)

    # Coupling Hamiltonian
    coup_op_t = 1j*transm.charge_zpf*(qtp.basis(lev_t, 1)*qtp.basis(lev_t, 0).dag() + np.sqrt(2)*qtp.basis(lev_t, 2)*qtp.basis(lev_t, 1).dag())
    q_10_f = flx.charge_op(as_qobj=True)[1, 0]
    q_30_f = flx.charge_op(as_qobj=True)[3, 0]
    q_21_f = flx.charge_op(as_qobj=True)[2, 1]
    coup_op_f = q_10_f*qtp.basis(lev_f, 1)*qtp.basis(lev_f, 0).dag() + \
        q_30_f*qtp.basis(lev_f, 3)*qtp.basis(lev_f, 0).dag() + q_21_f*qtp.basis(lev_f, 2)*qtp.basis(lev_f, 1).dag()
    h_coup = jc*(qtp.tensor(coup_op_t, coup_op_f.dag()) + qtp.tensor(coup_op_t, coup_op_f.dag()).dag())

    # Drive Hamiltonian
    h_drive = eps_drive/2*(qtp.tensor(id_t, coup_op_f).dag()*np.exp(1j*phase_drive) + \
                           qtp.tensor(id_t, coup_op_f)*np.exp(-1j*phase_drive))

    #Total Hamiltonian
    h_0 = h_t + h_f
    h = h_0 + h_coup
    h_tot = h + h_drive
    
    proj_dressed, states_dressed = projector(h, lev_t, lev_f)
    proj_dressed_drive, states_dressed_drive = projector(h_tot, lev_t, lev_f)
    ref_dressed = qtp.tensor(id_t, id_f) - 2*proj_dressed
    ref_dressed_drive = qtp.tensor(id_t, id_f) - 2*proj_dressed_drive
    u_sw = (ref_dressed*ref_dressed_drive).sqrtm()
    
    h_sw_eff = h_sw_exact(proj_dressed, proj_dressed_drive, h_tot, states_dressed, lev_t, lev_f)
    
    output = {}
    output["transmon_freq"] = transm.freq # GHz
    output["fluxonium_freq_01"] = eig_flx[1] - eig_flx[0] #GHz
    output["fluxonium_freq_12"] = eig_flx[2] - eig_flx[1] #GHz
    output["fluxonium_freq_03"] = eig_flx[3] - eig_flx[0] #GHz
    output["fluxonium_freq_23"] = eig_flx[3] - eig_flx[2] #GHz
    output["eps"] = eps_drive #GHz
    output["cr"] = np.real(get_pauli_coeff(h_sw_eff, "XZ")*1e3) #MHz
    output["cr_gate_time"] = 1/(8*np.abs(output["cr"])/1e3) #ns
    output["zz"] = 4*np.real(get_pauli_coeff(h_sw_eff, "ZZ")*1e3) #Mhz (notice factor of 4)
    output["jc"] = jc*1e3 #MHz
    
    return output

In [71]:
with open('flx_transm_params.txt') as param_file:
    PARAM_SETS = json.load(param_file)
    
PARAMS = PARAM_SETS["2"] # Choosing the second parameter set

In [84]:
TEMP = 0.020 # K
D_COMP = 4
LEVELS_TRANSMON = 3
LEVELS_FLUXONIUM = 4

thermal_energy = temperature_to_thermalenergy(TEMP) # kb T/h in GHz

#Transmon
transmon = qubits.SimpleTransmon(
    label='transmon', 
    max_freq=PARAMS["max_freq_t"], 
    anharm=PARAMS["anharm_t"],
    diel_loss_tan=PARAMS["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=LEVELS_TRANSMON,
    dephasing_times=PARAMS["dephasing_times_t"]
)

#Fluxonium
fluxonium = qubits.Fluxonium(
    label='fluxonium', 
    charge_energy=PARAMS["charge_energy_f"], 
    induct_energy=PARAMS["induct_energy_f"], 
    joseph_energy=PARAMS["joseph_energy_f"], #8.0, 
    diel_loss_tan=PARAMS["diel_loss_tan_f"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,
    dephasing_times=PARAMS["dephasing_times_f"] #ns/2*np.pi 
)
fluxonium.diagonalize_basis(LEVELS_FLUXONIUM)

coup_coeff = PARAMS["jc"]

In [73]:
%%timeit
get_params_sw(transmon, fluxonium, coup_coeff, {"freq": transmon.freq, "eps" : 0.6})

400 ms ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Transmon frequency scan

In [80]:
fluxonium_eig_energies = fluxonium.eig_energies()

fluxonium_freq_01 = fluxonium_eig_energies[1] - fluxonium_eig_energies[0]
fluxonium_freq_02 = fluxonium_eig_energies[2] - fluxonium_eig_energies[0]
fluxonium_freq_12 = fluxonium_eig_energies[2] - fluxonium_eig_energies[1]
fluxonium_freq_03 = fluxonium_eig_energies[3] - fluxonium_eig_energies[0]

In [91]:
NUM_FREQS = 1001
TRANSMON_FREQS = np.linspace(3.5, 7, NUM_FREQS)

# Debug

In [97]:
transmon = qubits.SimpleTransmon(
    label='transmon', 
    max_freq=4.0005, 
    anharm=PARAMS["anharm_t"],
    diel_loss_tan=PARAMS["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=LEVELS_TRANSMON,
    dephasing_times=PARAMS["dephasing_times_t"]
)

#Fluxonium
fluxonium = qubits.Fluxonium(
    label='fluxonium', 
    charge_energy=PARAMS["charge_energy_f"], 
    induct_energy=PARAMS["induct_energy_f"], 
    joseph_energy=PARAMS["joseph_energy_f"], #8.0, 
    diel_loss_tan=PARAMS["diel_loss_tan_f"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,
    dephasing_times=PARAMS["dephasing_times_f"] #ns/2*np.pi 
)
fluxonium.diagonalize_basis(LEVELS_FLUXONIUM)

coup_coeff = PARAMS["jc"]

In [104]:
drive_params = {"freq": transmon.freq, "eps" : 0.6}
jc = coup_coeff
transm = transmon
flx = fluxonium

freq_drive = drive_params["freq"]
eps_drive = drive_params["eps"]
phase_drive = np.pi/2

eig_flx = flx.eig_energies()

lev_t = 3
lev_f = 4
id_t = qtp.qeye(lev_t)
id_f = qtp.qeye(lev_f)
proj_1_t = qtp.basis(lev_t, 1)*qtp.basis(lev_t, 1).dag()
proj_2_t = qtp.basis(lev_t, 2)*qtp.basis(lev_t, 2).dag()
h_t_single = (transm.freq - freq_drive)*proj_1_t + (transm.anharm + 2*(transm.freq - freq_drive))*proj_2_t
h_t = qtp.tensor(h_t_single, id_f)

# Fluxonium Hamiltonian
proj_1_f = qtp.basis(lev_f, 1)*qtp.basis(lev_f, 1).dag()
proj_2_f = qtp.basis(lev_f, 2)*qtp.basis(lev_f, 2).dag()
proj_3_f = qtp.basis(lev_f, 3)*qtp.basis(lev_f, 3).dag()
ground_en_f = flx.eig_energies()[0]
h_f_single = flx.hamiltonian(as_qobj=True) - ground_en_f*id_f - freq_drive*proj_1_f - \
    freq_drive*proj_3_f - 2*freq_drive*proj_2_f
h_f = qtp.tensor(id_t, h_f_single)

# Coupling Hamiltonian
coup_op_t = 1j*transm.charge_zpf*(qtp.basis(lev_t, 1)*qtp.basis(lev_t, 0).dag() + np.sqrt(2)*qtp.basis(lev_t, 2)*qtp.basis(lev_t, 1).dag())
q_10_f = flx.charge_op(as_qobj=True)[1, 0]
q_30_f = flx.charge_op(as_qobj=True)[3, 0]
q_21_f = flx.charge_op(as_qobj=True)[2, 1]
q_32_f = flx.charge_op(as_qobj=True)[3, 2]
coup_op_f = q_10_f*qtp.basis(lev_f, 1)*qtp.basis(lev_f, 0).dag() + \
    q_30_f*qtp.basis(lev_f, 3)*qtp.basis(lev_f, 0).dag() + q_21_f*qtp.basis(lev_f, 2)*qtp.basis(lev_f, 1).dag()
#coup_op_f = q_10_f*qtp.basis(lev_f, 1)*qtp.basis(lev_f, 0).dag() + \
#    q_30_f*qtp.basis(lev_f, 3)*qtp.basis(lev_f, 0).dag() + q_21_f*qtp.basis(lev_f, 2)*qtp.basis(lev_f, 1).dag() + q_32_f*qtp.basis(lev_f, 3)*qtp.basis(lev_f, 2).dag()
    
h_0 = h_t + h_f
h_coup = jc*(qtp.tensor(coup_op_t, coup_op_f.dag()) + qtp.tensor(coup_op_t, coup_op_f.dag()).dag())
h = h_0 + h_coup
proj_dressed, states_dressed = projector(h, lev_t, lev_f)

ValueError: array must not contain infs or NaNs

In [108]:
dressed_state("01", h, lev_t, lev_f)[1]

ValueError: array must not contain infs or NaNs

In [120]:
eig_en

array([-4.33056445e+00, -4.03133292e+00, -4.03076072e+00, -3.71788253e+00,
       -3.41865352e+00, -3.41846024e+00, -3.00029233e-01, -1.68841266e-05,
        0.00000000e+00,  2.27398847e+00,  2.57400787e+00,  2.57402323e+00])

In [128]:
qtp.fidelity(state_tmp*state_tmp.dag(), state_0*state_0.dag())

0.0

In [143]:
A = state_tmp*state_tmp.dag()
sqrtmA = A.sqrtm()
B = state_0*state_0.dag()

In [147]:
A

Quantum object: dims = [[3, 4], [3, 4]], shape = (12, 12), type = oper, isherm = True
Qobj data =
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  7.36616793e-07  0.00000000e+00 -2.35636024e-06
  -8.58260274e-04  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.35636024e-06  0.00000000e+00  7.53775051e-06
   2.74548505e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -8.58260274e-04  0.00000000e+00  2.74548505e-03
   9.99991726e-01  0.00000000e+00  0.000

In [149]:
np.all(np.linalg.eigvals(A) >= 0)

True

In [158]:
from qutip.sparse import sp_eigs
import scipy.sparse as sp
from qutip import Qobj
evals, evecs = sp_eigs(A.data, A.isherm, sparse=False, tol=0, maxiter=100000)
numevals = len(evals)
dV = sp.spdiags(np.sqrt(evals, dtype=complex), 0, numevals, numevals, format='csr')
if A.isherm:
    spDv = dV.dot(evecs.T.conj().T)
else:
    spDv = dV.dot(np.linalg.inv(evecs.T))

out = Qobj(evecs.T.dot(spDv), dims=A.dims)

/Users/boris/.pyenv/versions/3.8.2/lib/python3.8/site-packages/qutip/sparse.py:29: RuntimeWarning: invalid value encountered in true_divide
  vec /= norm


In [170]:
evecs

array([[-0.00000000e+00 -0.j,  2.35636913e-06 -0.j, -0.00000000e+00 -0.j,
         9.99996231e-01 +0.j, -2.74549539e-03 -0.j, -0.00000000e+00 -0.j,
        -0.00000000e+00 -0.j, -0.00000000e+00 -0.j, -0.00000000e+00 -0.j,
        -0.00000000e+00 -0.j, -0.00000000e+00 -0.j, -0.00000000e+00 -0.j],
       [            nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj],
       [            nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj],
       [            nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,      

In [171]:
np.linalg.eig(A)

(array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([[-0.00000000e+00-0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          1.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 2.35636913e-06-0.j, -8.58263825e-04+0.j,  2.35636913e-06+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [-0.00000000e+00-0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  1.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
          0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 9.9999623

In [166]:
evecs

array([[-0.00000000e+00 -0.j,  2.35636913e-06 -0.j, -0.00000000e+00 -0.j,
         9.99996231e-01 +0.j, -2.74549539e-03 -0.j, -0.00000000e+00 -0.j,
        -0.00000000e+00 -0.j, -0.00000000e+00 -0.j, -0.00000000e+00 -0.j,
        -0.00000000e+00 -0.j, -0.00000000e+00 -0.j, -0.00000000e+00 -0.j],
       [            nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj],
       [            nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,             nan+nanj,             nan+nanj],
       [            nan+nanj,             nan+nanj,             nan+nanj,
                    nan+nanj,      

In [140]:
eig_en, eig_states = h.eigenstates()
    
lev_list = [int(index) for index in "00"]

if lev_list[0] >= lev_t or lev_list[1] >= lev_f:
    raise ValueError("Index error: indeces out of bound")
state_0 = qtp.tensor(qtp.basis(lev_t, lev_list[0]), qtp.basis(lev_f, lev_list[1]))
#en_0 = eig_en_0[state_index]
eig_states_list = list(eig_states)
state = eig_states_list[0]
en = eig_en[0]
fid = qtp.fidelity(state*state.dag(), state_0*state_0.dag())
for state_tmp, en_tmp in zip(eig_states_list, eig_en):
    fid_tmp = qtp.fidelity(state_tmp*state_tmp.dag(), state_0*state_0.dag())

/Users/boris/.pyenv/versions/3.8.2/lib/python3.8/site-packages/qutip/sparse.py:29: RuntimeWarning: invalid value encountered in true_divide
  vec /= norm


ValueError: array must not contain infs or NaNs

In [98]:
get_params_sw(transmon, fluxonium, coup_coeff, {"freq": transmon.freq, "eps" : 0.6})

ValueError: array must not contain infs or NaNs

In [95]:
results = np.zeros((NUM_FREQS, 10), dtype=float)

for i, freq in enumerate(TRANSMON_FREQS):
    transmon = qubits.SimpleTransmon(
        label='transmon', 
        max_freq=freq, 
        anharm=PARAMS["anharm_t"],
        diel_loss_tan=PARAMS["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
        env_thermal_energy=thermal_energy,    
        dim_hilbert=LEVELS_TRANSMON,
        dephasing_times=PARAMS["dephasing_times_t"]
    )
    drive_params = dict(freq = transmon.freq, eps = 0.6)

    params = get_params_sw(transmon, fluxonium, coup_coeff, drive_params)
    results[i] = np.array(list(params.values()))  

ValueError: array must not contain infs or NaNs

# Full parameter scan

In [52]:
MAX_RESIST_STD = 6e-2
MAX_RESIST_OFFSET = 4 * MAX_RESIST_STD # Corresponding to 4 sigmas from the gaussian distribtuion

NUM_POINTS = 101
RESIST_OFFSETS = np.linspace(-MAX_RESIST_OFFSET, MAX_RESIST_OFFSET, NUM_POINTS)

In [57]:
TRANSMON_TARGET_FREQ = 4.4
FREQ_OFFSETS = 0.5 * RESIST_OFFSETS * TRANSMON_TARGET_FREQ

TARGET_JOSEPH_ENERGY = PARAMS["joseph_energy_f"]
JOSEPH_ENERGY_OFFSETS = RESIST_OFFSETS * TARGET_JOSEPH_ENERGY

TARGET_INDUCT_ENERGY = PARAMS["induct_energy_f"]
NUM_JUNCTIONS = 41
INDUCT_ENERGY_OFFSETS = RESIST_OFFSETS * TARGET_INDUCT_ENERGY / np.sqrt(NUM_JUNCTIONS)

In [58]:
results = np.zeros((NUM_POINTS, NUM_POINTS, NUM_POINTS, 10))

for i, freq_offset in enumerate(FREQ_OFFSETS):
    for j, joseph_energy_offset in enumerate(JOSEPH_ENERGY_OFFSETS):
        for k, induct_energy_offset in enumerate(INDUCT_ENERGY_OFFSETS):
            transmon.max_freq = TRANSMON_TARGET_FREQ + freq_offset
            
            fluxonium.joseph_energy = TARGET_JOSEPH_ENERGY + joseph_energy_offset
            fluxonium.induct_energy = TARGET_INDUCT_ENERGY + induct_energy_offset
            
            drive_params = dict(freq = transmon.freq, eps = 0.6)
            
            params = get_params_sw(transmon, fluxonium, coup_coeff, drive_params)
            results[i, j, k] = np.array(list(params.values()))  

/Users/boris/.pyenv/versions/3.8.2/lib/python3.8/site-packages/qutip/sparse.py:29: RuntimeWarning: invalid value encountered in true_divide
  vec /= norm


ValueError: array must not contain infs or NaNs

In [60]:
params

{'transmon_freq': 3.9036800000000005,
 'fluxonium_freq_01': 0.41345106999796877,
 'fluxonium_freq_12': 3.7915125826915674,
 'fluxonium_freq_03': 6.605856575045441,
 'fluxonium_freq_23': 2.4008929223559043,
 'eps': 0.6,
 'cr': -5.162745692139531,
 'cr_gate_time': 24.211922774022568,
 'zz': 0.6016950240925745,
 'jc': 20.0}

In [59]:
results

array([[[[3.87200000e+00, 9.05230607e-01, 2.97699808e+00, ...,
          1.42134336e+02, 1.19553209e-01, 2.00000000e+01],
         [3.87200000e+00, 9.06098965e-01, 2.97709049e+00, ...,
          1.42169453e+02, 1.19544545e-01, 2.00000000e+01],
         [3.87200000e+00, 9.06967447e-01, 2.97718357e+00, ...,
          1.42204639e+02, 1.19535848e-01, 2.00000000e+01],
         ...,
         [3.87200000e+00, 9.90891665e-01, 2.98907585e+00, ...,
          1.45922210e+02, 1.18541252e-01, 2.00000000e+01],
         [3.87200000e+00, 9.91771227e-01, 2.98922923e+00, ...,
          1.45964595e+02, 1.18529195e-01, 2.00000000e+01],
         [3.87200000e+00, 9.92650891e-01, 2.98938319e+00, ...,
          1.46007058e+02, 1.18517104e-01, 2.00000000e+01]],

        [[3.87200000e+00, 8.96700274e-01, 2.98321022e+00, ...,
          1.40246813e+02, 1.20949007e-01, 2.00000000e+01],
         [3.87200000e+00, 8.97564178e-01, 2.98329449e+00, ...,
          1.40281317e+02, 1.20940234e-01, 2.00000000e+01],
        

In [61]:
get_params_sw(transmon, fluxonium, coup_coeff, drive_params)

ValueError: array must not contain infs or NaNs